# Introduction to Sparse Matrices with SciPy

*A visual and practical tutorial for engineering students*

---

## 1. Why sparse matrices appear in engineering problems

In engineering (fluid networks, electrical circuits, FEM, heat transfer), we often solve systems like:

$$
\mathbf{K} \mathbf{x} = \mathbf{b}
$$

Where:

* $\mathbf{K}$: global system matrix
* $\mathbf{x}$: unknown nodal variables
* $\mathbf{b}$: source or load vector

In **network-based problems**, each node interacts only with a few neighbors.
Therefore, most entries of $\mathbf{K}$ are zero.

➡️ This leads naturally to **sparse matrices**.

---

## 2. Dense vs sparse matrices (conceptual example)

### Dense matrix (what NumPy stores by default)

In [1]:
import numpy as np

A_dense = np.array([
    [10, -10,   0,   0],
    [-10,  20, -10,  0],
    [0,   -10,  20, -10],
    [0,     0, -10, 10]
])

print(A_dense)

[[ 10 -10   0   0]
 [-10  20 -10   0]
 [  0 -10  20 -10]
 [  0   0 -10  10]]


All entries (including zeros) are stored in memory.

---

### Sparse matrix (what SciPy stores)

In [2]:
from scipy import sparse

A_sparse = sparse.lil_matrix((4, 4))

At this point:

* Matrix size is (4 \times 4)
* **No values are stored yet**
* Memory usage is minimal

## 3. Building a sparse matrix step by step

Let us assemble the same matrix incrementally.

In [3]:
A = sparse.lil_matrix((4, 4))

A[0, 0] += 10
A[0, 1] -= 10

A[1, 0] -= 10
A[1, 1] += 20
A[1, 2] -= 10

A[2, 1] -= 10
A[2, 2] += 20
A[2, 3] -= 10

A[3, 2] -= 10
A[3, 3] += 10

This is exactly what happens during **network or FEM assembly**.

---

## 4. Printing sparse matrices (important!)

If we print a sparse matrix directly:

In [4]:
print(A)

<List of Lists sparse matrix of dtype 'float64'
	with 10 stored elements and shape (4, 4)>
  Coords	Values
  (0, 0)	10.0
  (0, 1)	-10.0
  (1, 0)	-10.0
  (1, 1)	20.0
  (1, 2)	-10.0
  (2, 1)	-10.0
  (2, 2)	20.0
  (2, 3)	-10.0
  (3, 2)	-10.0
  (3, 3)	10.0


You will see something like:

```
<List of Lists sparse matrix of dtype 'float64'
    with 10 stored elements and shape (4, 4)>
```

This does **not** show the matrix values.

---

### Convert sparse → dense (for visualization only!)

In [5]:
A_dense_from_sparse = A.toarray()
print(A_dense_from_sparse)

[[ 10. -10.   0.   0.]
 [-10.  20. -10.   0.]
 [  0. -10.  20. -10.]
 [  0.   0. -10.  10.]]


⚠️ **Important**:
Converting to dense is only for:

* Teaching
* Debugging
* Very small problems

Never do this for large systems.

---

## 5. Sparse matrix formats (LIL → CSR)

### LIL format (assembly)

In [19]:
A_lil = sparse.lil_matrix((6, 6))

A_lil[0, 0] += 10
A_lil[0, 1] -= 10

A_lil[1, 0] -= 10
A_lil[1, 1] += 25
A_lil[1, 2] -= 10
A_lil[1, 3] -= 5

A_lil[2, 1] -= 10
A_lil[2, 2] += 20
A_lil[2, 4] -= 10

A_lil[3, 1] -= 5
A_lil[3, 3] += 15
A_lil[3, 5] -= 10

A_lil[4, 2] -= 10
A_lil[4, 4] += 20
A_lil[4, 5] -= 10

A_lil[5, 3] -= 10
A_lil[5, 4] -= 10
A_lil[5, 5] += 20



* Efficient for inserting values
* Poor performance for solving

In [20]:
A_dense = A_lil.toarray()
print(A_dense)

[[ 10. -10.   0.   0.   0.   0.]
 [-10.  25. -10.  -5.   0.   0.]
 [  0. -10.  20.   0. -10.   0.]
 [  0.  -5.   0.  15.   0. -10.]
 [  0.   0. -10.   0.  20. -10.]
 [  0.   0.   0. -10. -10.  20.]]


### CSR format (solving)

In [21]:
A_csr = A_lil.tocsr()

* Efficient matrix–vector products
* Required by most solvers

Your framework **always assembles in LIL and solves in CSR**.

---

## 6. Sparse slicing and matrix partitioning

In network problems, some nodes are **known** (Dirichlet conditions).

Suppose:

In [22]:
known = np.array([0, 5])
unknown = np.array([1, 2, 3, 4])

### Matrix partitioning

In [23]:
K = A_csr

K_uu = K[unknown, :][:, unknown]
K_uk = K[unknown, :][:, known]

### Mathematical meaning

The full system is:

$$
\begin{bmatrix}
K_{kk} & K_{ku} \\
K_{uk} & K_{uu}
\end{bmatrix}
\begin{bmatrix}
\mathbf{x}_k \\
\mathbf{x}_u
\end{bmatrix}
= \begin{bmatrix}
\mathbf{b}_k \\
\mathbf{b}_u
\end{bmatrix}
$$

We solve:

$$
K_{uu},\mathbf{x}_u
= \mathbf{b}_u - K_{uk},\mathbf{x}_k
$$

---

### Visualizing the submatrices

In [26]:
print("K_uu:")
print(K_uu.toarray())

K_uu:
[[ 25. -10.  -5.   0.]
 [-10.  20.   0. -10.]
 [ -5.   0.  15.   0.]
 [  0. -10.   0.  20.]]


In [27]:
print("K_uk:")
print(K_uk.toarray())

K_uk:
[[-10.   0.]
 [  0.   0.]
 [  0. -10.]
 [  0. -10.]]


Sparse slicing behaves **exactly like NumPy slicing**, but keeps sparsity.

---

## 7. Sparse matrix–vector multiplication

Consider:

In [28]:
x_k = np.array([100.0, 50.0])  # nodes 0 and 5
b_u = np.zeros(len(unknown))

Compute the right-hand side

In [29]:
rhs = b_u - K_uk @ x_k
print(rhs)

[1000.    0.  500.  500.]


### Mathematical interpretation

This corresponds to:

$$
\mathbf{r} = \mathbf{b}_u - K_{uk}\mathbf{x}_k
$$

Each matrix–vector multiplication is:

$$
(K_{uk}\mathbf{x}_k)_i
=\sum_j K_{ij}x_j
$$

Sparse matrices:

* Skip zero entries
* Perform only necessary multiplications

---

## 8. Solving a sparse linear system

In [30]:
from scipy.sparse.linalg import spsolve

x_u = spsolve(K_uu, rhs)
print("Unknown nodal values:")
print(x_u)

Unknown nodal values:
[80. 70. 60. 60.]


This solves:

$$
K_{uu},\mathbf{x}_u = \mathbf{r}
$$

---

## 9. Relation to your network framework

In your code:

* Each **element** contributes a local matrix
* Assembly uses **sparse LIL**
* Boundary conditions use **sparse slicing**
* Solvers use **CSR matrices**
* Outputs are **nodal vectors**

This is exactly the same methodology used in:

* FEM solvers
* CFD codes
* Electrical circuit simulators
* Multiphysics frameworks

---

## 10. Why this approach scales

| Nodes   | Dense matrix | Sparse matrix |
| ------- | ------------ | ------------- |
| 100     | OK           | OK            |
| 1,000   | Slow         | Fast          |
| 10,000  | Impossible   | Feasible      |
| 100,000 | ❌            | ✅             |

Sparse matrices are **not optional** in real engineering solvers.

---

## 11. Key takeaways 

* Sparse matrices store **only what matters**
* Assembly → LIL
* Solving → CSR
* Sparse slicing = mathematical partitioning
* Operations correspond directly to equations

---

## 12. Engineering mindset

> Sparse matrices allow us to translate physical connectivity
> **directly into efficient numerical algorithms**.